# Sondera Harness + Google ADK

**Add policy enforcement to your Google ADK agents.**

This notebook shows how to use Sondera Harness with Google's Agent Development Kit (ADK). You'll define ADK tools, write Cedar policies, and see how the harness evaluates each action before it executes.

We'll build a customer service agent that:
- **Allows** looking up customer information
- **Allows** processing refunds up to \$10,000
- **Blocks** refunds over \$10,000 (requires supervisor approval)
- **Blocks** accessing internal system commands

*This is a simulation: we evaluate what would be allowed, but no actual LLM calls are made.*

## Install

In [ ]:
%%capture
%pip install -U protobuf
%pip install "sondera-harness[adk]"

## Define ADK Tools

In ADK, tools are just Python functions. ADK automatically extracts parameter types and descriptions from type hints and docstrings.

In [ ]:
def get_customer_info(customer_id: str) -> str:
    """Look up customer information by their ID."""
    return f"Customer {customer_id}: John Doe, Premium Member since 2020"


def get_order_history(customer_id: str, limit: int = 5) -> str:
    """Get recent orders for a customer."""
    return f"Last {limit} orders for {customer_id}: Order #1001, #1002, #1003"


def process_refund(customer_id: str, order_id: str, amount: int, reason: str) -> str:
    """Process a refund for a customer's order. Amount in dollars."""
    return f"Refund of ${amount} processed for order {order_id}"


def escalate_to_supervisor(customer_id: str, issue: str) -> str:
    """Escalate an issue to a human supervisor."""
    return f"Issue escalated for {customer_id}: {issue}"


tools = [get_customer_info, get_order_history, process_refund, escalate_to_supervisor]
print(f"Defined {len(tools)} tools: {[t.__name__ for t in tools]}")

## Create the ADK Agent

Create a Google ADK agent with your tools. We'll then extract metadata from this agent for policy evaluation.

In [ ]:
from google.adk import Agent

from sondera.adk.analyze import format as adk_to_metadata

# Create the ADK agent
adk_agent = Agent(
    name="customer_service",  # Note: ADK requires underscores, not hyphens
    model="gemini-2.0-flash",
    instruction="You are a helpful customer service agent. Help customers with orders, refunds, and account questions.",
    description="Customer service agent for order and refund inquiries.",
    tools=tools,
)

# Extract metadata for policy evaluation
agent_metadata = adk_to_metadata(adk_agent)

print(f"ADK agent '{adk_agent.name}' created with {len(tools)} tools")
print("Metadata extracted for policy evaluation:")
for tool in agent_metadata.tools:
    print(f"  - {tool.name}: {len(tool.parameters)} parameters")

## Define the Policy

Write Cedar policies to control what the agent can do. The namespace `customer_service` matches the ADK agent name.

**Pattern:** Permit by default, then forbid specific things. `forbid` always wins over `permit`.

In [ ]:
from cedar import PolicySet

policy_set = PolicySet("""
// Allow looking up customer information
@id("allow-get-customer-info")
permit(principal, action == customer_service::Action::"get_customer_info", resource);

// Allow viewing order history
@id("allow-get-order-history")
permit(principal, action == customer_service::Action::"get_order_history", resource);

// Allow escalation to supervisors
@id("allow-escalate")
permit(principal, action == customer_service::Action::"escalate_to_supervisor", resource);

// Allow refunds by default (restricted below)
@id("allow-process-refund")
permit(principal, action == customer_service::Action::"process_refund", resource);

// Block refunds over $10,000 - requires supervisor approval
@id("forbid-large-refund")
forbid(principal, action == customer_service::Action::"process_refund", resource)
when {
  context has parameters &&
  context.parameters.amount > 10000
};
""")

print(f"Policy loaded with {len(policy_set)} rules")

## Initialize the Harness

Create a `CedarPolicyHarness` for local policy evaluation. In production, you'd use `SonderaRemoteHarness` with `SonderaHarnessPlugin` to enforce policies automatically.

In [ ]:
from sondera import CedarPolicyHarness
from sondera.harness.cedar.schema import agent_to_cedar_schema

schema = agent_to_cedar_schema(agent_metadata)
harness = CedarPolicyHarness(policy_set=policy_set, schema=schema)
await harness.initialize(agent=agent_metadata)

print("Harness initialized and ready")

## How the Plugin Works

In a real ADK agent, `SonderaHarnessPlugin` intercepts execution at each stage:

| ADK Callback | Stage | What It Checks |
|:-------------|:------|:---------------|
| `on_before_agent` | `PRE_RUN` | Session start, user input |
| `on_before_model` | `PRE_MODEL` | Model requests |
| `on_after_model` | `POST_MODEL` | Model responses |
| `on_before_tool` | `PRE_TOOL` | Tool arguments |
| `on_after_tool` | `POST_TOOL` | Tool results |
| `on_after_agent` | `POST_RUN` | Session end, finalize trajectory |

Below, we'll simulate what the plugin does by calling `adjudicate()` directly.

## Run It

Let's test what the agent is allowed to do. We call `adjudicate()` for each action, simulating what the plugin would do at the `PRE_TOOL` stage.

In [ ]:
from sondera import Decision, Role, Stage, ToolRequestContent

# ANSI color codes
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"


def check(description: str, result, expected: Decision):
    """Check result matches expected and print colored status."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    status = result.decision.name
    icon = "\u2705" if result.decision == expected else "\u274c"
    print(f"{icon} {description} ({color}{status}{RESET})")
    if result.decision != expected:
        print(f"   Expected {expected.name}, got {result.decision.name}")
        if result.reason:
            print(f"   Reason: {result.reason}")


# =============================================================================
# ALLOWED OPERATIONS
# =============================================================================

# Looking up customer info is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="get_customer_info", args={"customer_id": "CUST001"}),
)
check("Look up customer info", result, Decision.ALLOW)

# Viewing order history is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="get_order_history", args={"customer_id": "CUST001", "limit": 10}
    ),
)
check("View order history", result, Decision.ALLOW)

# Small refund is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "order_id": "ORD1001",
            "amount": 250,
            "reason": "Item arrived damaged",
        },
    ),
)
check("Process $250 refund", result, Decision.ALLOW)

# Refund at the limit is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "order_id": "ORD1002",
            "amount": 10000,
            "reason": "Order never arrived",
        },
    ),
)
check("Process $10,000 refund (at limit)", result, Decision.ALLOW)

# Escalation is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_to_supervisor",
        args={
            "customer_id": "CUST001",
            "issue": "Customer requesting exception to policy",
        },
    ),
)
check("Escalate to supervisor", result, Decision.ALLOW)

# =============================================================================
# BLOCKED OPERATIONS
# =============================================================================

# Large refund is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "order_id": "ORD1003",
            "amount": 15000,
            "reason": "Bulk order cancellation",
        },
    ),
)
check("Process $15,000 refund (over limit)", result, Decision.DENY)

# Very large refund is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "order_id": "ORD1004",
            "amount": 50000,
            "reason": "Major service failure",
        },
    ),
)
check("Process $50,000 refund (way over limit)", result, Decision.DENY)

print(f"\n{GREEN}{'=' * 50}")
print(f"All policy evaluations complete!{RESET}")

## Using the Plugin

In a real ADK agent, you'd use `SonderaHarnessPlugin` to automatically enforce policies. Here's how:

```python
from google.adk import Agent
from google.adk.runners import Runner
from sondera.harness import SonderaRemoteHarness
from sondera.adk import SonderaHarnessPlugin

# Create your ADK agent
agent = Agent(
    name="customer_service",
    model="gemini-2.0-flash",
    instruction="Be helpful.",
    tools=[...],
)

# Create harness (uses SONDERA_API_TOKEN from environment)
harness = SonderaRemoteHarness()

# Create plugin
plugin = SonderaHarnessPlugin(harness=harness)

# Create runner with plugin
runner = Runner(
    agent=agent,
    app_name="my-app",
    plugins=[plugin],
)

# Run the agent - policies are automatically enforced
async for event in runner.run_async(user_id="user-1", session_id="session-1"):
    print(event)
```

**On denial:** The plugin returns an error message to the agent, allowing it to inform the user or try a different approach.

## Try It Yourself

Experiment with the policy. Go back to **Define the Policy** and try:

- **Change the refund limit**: Try `context.parameters.amount > 5000` instead of `10000`
- **Block specific customers**: Add a rule that forbids refunds when `context.parameters.customer_id == "BLOCKED_USER"`
- **Require escalation for large refunds**: Instead of blocking, require that `escalate_to_supervisor` is called first

Or modify the test cases below:

In [ ]:
def show(description: str, result):
    """Print colored result."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    print(f"{description}: {color}{result.decision.name}{RESET}")


# Try a refund right at the boundary
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "order_id": "ORD2001",
            "amount": 10001,  # Just over the limit
            "reason": "Edge case test",
        },
    ),
)
show("Process $10,001 refund", result)

# Try with a different customer
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="get_customer_info", args={"customer_id": "VIP_CUSTOMER"}
    ),
)
show("Look up VIP customer", result)

# Try an escalation with detailed issue
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="escalate_to_supervisor",
        args={
            "customer_id": "CUST001",
            "issue": "Customer requesting $50,000 refund - needs supervisor approval per policy",
        },
    ),
)
show("Escalate large refund request", result)

## Next Steps

- [Quickstart](https://docs.sondera.ai/quickstart/) - Core concepts without framework integration
- [Writing Policies](https://docs.sondera.ai/writing-policies/) - Cedar syntax and patterns
- [LangGraph Integration](https://docs.sondera.ai/integrations/langgraph/) - Use with LangChain and LangGraph
- [Strands Integration](https://docs.sondera.ai/integrations/strands/) - Use with AWS Strands